In [1]:
# imports
from z3 import *
from utils import *
import numpy as np

In [2]:
inst01 = {
    'm': 2,
    'n': 6,
    'l': [15, 10],
    's': [3, 2, 6, 5, 4, 4],
    'min_dist': 4,
    'max_dist': 40,
    'time': 5,
    'at_least_one': 1,
    'distances': [[0, 3, 4, 5, 6, 6, 2], [3, 0, 1, 4, 5, 7, 3], [4, 1, 0, 5, 6, 6, 4], [4, 4, 5, 0, 3, 3, 2], [6, 7, 8, 3, 0, 2, 4], [6, 7, 8, 3, 2, 0, 4], [2, 3, 4, 3, 4, 4, 0]]
}

inst02 = {
    'm': 6,
    'n': 9,
    'l': [190, 185, 185, 190, 195, 185],
    's': [11, 11, 23, 16, 2, 1, 24, 14, 20],
    'min_dist': 64,
    'max_dist': 1175,
    'time': 5,
    'at_least_one': 1,
    'distances': [[0, 199, 119, 28, 179, 77, 145, 61, 123, 87], [199, 0, 81, 206, 38, 122, 55, 138, 76, 113], [119, 81, 0, 126, 69, 121, 26, 117, 91, 32], [28, 206, 126, 0, 186, 84, 152, 68, 130, 94], [169, 38, 79, 176, 0, 92, 58, 108, 46, 98], [77, 122, 121, 84, 102, 0, 100, 16, 46, 96], [145, 55, 26, 152, 58, 100, 0, 91, 70, 58], [61, 138, 113, 68, 118, 16, 91, 0, 62, 87], [123, 76, 91, 130, 56, 46, 70, 62, 0, 66], [87, 113, 32, 94, 94, 96, 58, 87, 66, 0]]
}

inst04 = {
    'm': 8,
    'n': 10,
    'l': [200, 180, 160, 185, 180, 200, 160, 200],
    's': [10, 25, 18, 16, 14, 16, 24, 19, 23, 19],
    'min_dist': 34,
    'max_dist': 690,
    'time': 4,
    'at_least_one': 1,
    'distances': [[0, 56, 86, 77, 81, 128, 107, 154, 70, 93, 53], [56, 0, 79, 31, 62, 87, 61, 107, 37, 37, 24], [86, 79, 0, 109, 17, 43, 110, 68, 43, 69, 55], [87, 31, 109, 0, 92, 116, 30, 77, 66, 40, 55], [81, 62, 17, 92, 0, 47, 93, 82, 26, 52, 38], [128, 87, 43, 116, 47, 0, 117, 52, 58, 76, 75], [116, 61, 110, 30, 93, 117, 0, 65, 67, 41, 63], [163, 107, 78, 77, 82, 52, 65, 0, 93, 70, 110], [70, 41, 43, 66, 26, 73, 67, 93, 0, 26, 17], [93, 37, 69, 40, 52, 76, 41, 70, 26, 0, 40], [53, 24, 55, 54, 38, 75, 63, 110, 17, 40, 0]]
}

In [3]:
def min_index(array):
    for i in range(len(array)):
        if array[i] == True:
            return i
    return 0

def max_index(array):
    for i in range(len(array)-1, -1, -1):
        if array[i] == True:
            return i
    return len(array)

def indeces(array):
    indeces = []
    indeces.append(len(array))
    for i in range(len(array)-1, -1, -1):
        if array[i] == True:
            indeces.append(i)
    return indeces


In [4]:
def MCP(instance):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of couriers
    time = instance["time"] # time
    at_least_one = instance["at_least_one"]
    equal_load_matrix = instance["equal_matrix"]

    solver = Solver()

    # Variables
    # To codify that courier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(time+1)] for j in range (n+1)] for i in range(m)]

    # To codify that courier i goes from city start to city end
    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)]
          for start in range(n+1)] for i in range(m)]

    for i in range(m):
        for k in range(time):
            for startj in range(n+1):
                for endj in range(n+1):
                    solver.add(
                        Implies(And(v[i][startj][k], v[i][endj][k+1]), d[i][startj][endj])
                    )

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range (m):
        weigth_set = []
        for j in range (n):
            for k in range(1,time):
                for _ in range(s[j]):
                    weigth_set.append(v[i][j][k])
        solver.add(at_most_k_seq(weigth_set, l[i], f"courier_{i}_load"))
    

    # 2. Each courier i starts and end at position j = n
    for i in range (m):
        solver.add(And(v[i][n][0], v[i][n][time]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(time+1):
            solver.add(exactly_one_bw([v[i][j][k] for j in range(n+1)], f"amo_package_{i}_{k}"))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([v[i][j][k] for k in range(1,time) for i in range(m)], f"exactly_once_{j}"))
    
    # 5. If we know that each courier must deliver at least one package
    if at_least_one == 1:
        for i in range(m):
            solver.add(Or([v[i][j][1] for j in range(n)]))

    # Symmetry breaking constraints
    # 6. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, time-1):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # symmetry breaking for courriers with equal capacity
    # 7. if the courrier i have lower capacity than currier i1, then what can be delivered by i cant be delivered by i1
    #if equal_load_matrix[i][i1] is true, then v[i][:][k] should be lexicographically less than v[i1][:][k]
    assignments_array = [[Bool(f"aa_{i}_{j}") for j in range(n)] for i in range (m)]

    # aa true if the package j is assigned to i at any k
    for i in range(m):
        for j in range(n):
            solver.add(
                assignments_array[i][j] == Or([v[i][j][k] for k in range(1,time)]) 
            )

    for i in range(m):
        for i1 in range(i+1,m):
            for j in range(1,min_index([assignments_array[i][jj] for jj in range(n)])+1):
                solver.add(
                    Implies(
                        equal_load_matrix[i][i1],
                        Implies( Or(And(assignments_array[i][j-1], assignments_array[i1][j-1]), And(Not(assignments_array[i][j-1]),Not(assignments_array[i1][j-1]))) ,
                            Or(And(assignments_array[i][j], assignments_array[i1][j]), And(Not(assignments_array[i][j]),Not(assignments_array[i1][j])), assignments_array[i][j])
                        )
                    )
                )

    return solver, v, d

def add_distance_constraint(solver, instance, v, d, upperBound):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    time = instance["time"] # time
    distances = instance["distances"] # distances between packages
    for i in range(m):
        dists = []
        for start in range(n+1):
            for end in range(n+1):
                for z in range(distances[start][end]):
                    dists.append(d[i][start][end])
        solver.add(at_most_k_seq(dists, upperBound, f"Courier_dist_{i}_{upperBound}"))
    
    flattened_distances = np.array(instance['distances']).flatten()
    flattened_distances = flattened_distances[flattened_distances != 0]
    flattened_distances = np.sort(flattened_distances)

    max_k = 0

    while np.sum(flattened_distances[:max_k]) <= upperBound and max_k < n:
        max_k += 1

    print('max_k', max_k)

    if max_k < time:
        for i in range(m):
            solver.add(v[i][n][max_k + 1])

In [5]:
# Second version using Pseudo-Boolean constraints
def MCP_Pb(instance):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    l = instance["l"] # weigths
    s = instance["s"] # sizes of courriers
    time = instance["time"] # time
    at_least_one = instance["at_least_one"] # if each courrier must deliver at least one package
    equal_load_matrix = instance["equal_matrix"]

    solver = Solver()

    # To codify that courrier i deliver package j at time k
    v = [[[Bool(f"x_{i}_{j}_{k}") for k in range(time+1)] for j in range (n+1)] for i in range(m)]

    d = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n+1)]
          for start in range(n+1)] for i in range(m)]

    for i in range(m):
        for k in range(time):
            for startj in range(n+1):
                for endj in range(n+1):
                    solver.add(
                        Implies(And(v[i][startj][k], v[i][endj][k+1]), d[i][startj][endj])
                    )

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    # Pb version
    for i in range(m):
       solver.append(PbLe([(v[i][j][k],s[j]) for j in range(n) for k in range(1,time)], l[i]))


    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(And(v[i][n][0], v[i][n][time]))


    # 3. Each courier can't be in two places at the same time 
    for i in range(m):
        for k in range(time+1):
            solver.add(exactly_one_seq([v[i][j][k] for j in range(n+1)], f"amo_package_{k}_{i}")) #(PbEq([(v[i][j][k],1) for j in range(n+1)],1))
    
    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_seq([v[i][j][k] for k in range(1,time) for i in range(m)], f"exactly_once_{j}")) #(PbEq([(v[i][j][k],1) for k in range(1,n+1) for i in range(m)],1))    
    
    # 5. If we know that each courier must deliver at least one package
    if at_least_one == 1:
        for i in range(m):
            solver.add(Or([v[i][j][1] for j in range(n)]))

    # Symmetry breaking constraints
    # 6. once a courier arrive to depot (j = n+1), it can't depart from there
    for i in range (m):
        for k in range(1, time-1):
            solver.add(
                Implies(v[i][n][k], v[i][n][k+1])
            )
    
    # 7. symmetry breaking for courriers with equal capacity
    assignments_array = [[Bool(f"aa_{i}_{j}") for j in range(n)] for i in range (m)]

    # aa true if the package j is assigned to i at any k
    for i in range(m):
        for j in range(n):
            solver.add(
                assignments_array[i][j] == Or([v[i][j][k] for k in range(1,time)]) 
            )

    for i in range(m):
        for i1 in range(i+1,m):
            for j in range(1,min_index([assignments_array[i][jj] for jj in range(n)])+1):
                solver.add(
                    Implies(
                        equal_load_matrix[i][i1],
                        Implies( Or(And(assignments_array[i][j-1], assignments_array[i1][j-1]), And(Not(assignments_array[i][j-1]),Not(assignments_array[i1][j-1]))) ,
                            Or(And(assignments_array[i][j], assignments_array[i1][j]), And(Not(assignments_array[i][j]),Not(assignments_array[i1][j])), assignments_array[i][j])
                        )
                    )
                )

    return solver, v, d

def add_distance_constraint_pb(solver, instance, v, d, upperBound):
    m = instance["m"] # courriers
    n = instance["n"] # packages
    time = instance["time"] # time
    distances = instance["distances"] # distances between packages

    # distance calculation
    # Pb version
    for i in range(m):
        solver.append(PbLe([(d[i][j1][j2],distances[j1][j2]) for j1 in range (n+1) for j2 in range (n+1)], upperBound))
    
    flattened_distances = np.array(instance['distances']).flatten()
    flattened_distances = flattened_distances[flattened_distances != 0]
    flattened_distances = np.sort(flattened_distances)

    max_k = 0

    while np.sum(flattened_distances[:max_k]) <= upperBound and max_k < n:
        max_k += 1

    if max_k < time:
        for i in range(m):
            solver.add(v[i][n][max_k + 1])

In [6]:
def solve_mcp(solver, instance, v, verbose=False):
    m = instance["m"] # couriers
    n = instance["n"] # packages
    time = instance["time"] # time
    if not verbose:
        if solver.check() == sat:
            return solver.model()
        else:  
            return False
    else:
        print(solver.check())
        if solver.check() == sat:
            model = solver.model()
            for i in range(m):
                print()
                print(f"Courier {i+1}:")
                for k in range(time+1):
                    for j in range(n+1):
                        if model[v[i][j][k]]:
                            print(f"Time {k} Place {j} : {model[v[i][j][k]]}")
            return solver.model()
        else:
            return False

In [7]:
def MultipleCouriersPlanning(instance, usePb = False):
    if usePb:
        mcp = MCP_Pb
        add_constraint = add_distance_constraint_pb
    else:
        mcp = MCP
        add_constraint = add_distance_constraint

    m = instance["m"]
    l = instance["l"]
    equal_matrix = np.full((m,m), Bool(False))
    for i in range(m):
        for i1 in range(m):
            if l[i] == l[i1]:
                equal_matrix[i,i1] = True
    instance["equal_matrix"] = equal_matrix

    originalUpperBound = instance["max_dist"]
    lowerBound = instance["min_dist"]
    upperBound = originalUpperBound

    pivot = (originalUpperBound + lowerBound) // 2

    solver, v, d = mcp(instance) # create basic solver

    # binary search using bounds
    print("searching...")
    while True:
        if lowerBound == upperBound:
            return upperBound
        print("pivot:",pivot)

        solver.push()
        add_constraint(solver, instance, v, d, pivot)

        res = solve_mcp(solver, instance, v, verbose)
        if(res == False):
            print("fail")
            if lowerBound > upperBound:
                if upperBound == originalUpperBound:
                    return unsat
                else:
                    return upperBound
            lowerBound = pivot + 1

            solver.pop()

            pivot = (upperBound + lowerBound) // 2
        else:
            print("success")
            upperBound = pivot
            pivot = (upperBound + lowerBound) // 2

In [10]:
usePb = True
verbose = True
instance = inst04
result = MultipleCouriersPlanning(instance, usePb)
if result == unsat:
    print("\nResult of the computation: unsat")
else:    
    print("\nResult of the minimization: ",result)

searching...
pivot: 362
sat

Courier 1:
Time 0 Place 10 : True
Time 1 Place 6 : True
Time 2 Place 8 : True
Time 3 Place 9 : True
Time 4 Place 10 : True

Courier 2:
Time 0 Place 10 : True
Time 1 Place 7 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 3:
Time 0 Place 10 : True
Time 1 Place 5 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 4:
Time 0 Place 10 : True
Time 1 Place 4 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 5:
Time 0 Place 10 : True
Time 1 Place 3 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 6:
Time 0 Place 10 : True
Time 1 Place 2 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 7:
Time 0 Place 10 : True
Time 1 Place 1 : True
Time 2 Place 10 : True
Time 3 Place 10 : True
Time 4 Place 10 : True

Courier 8:
Time 0 Place 10 : True
Time 1 Place 0 : True
Time 2 Place 10 : True
Time 3 Place